# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
# DEPLOYMENT_URL="http://localhost:2024/"
DEPLOYMENT_URL="http://localhost:8000/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data
 
 
def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)
 
        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    if msg_type == "ai":
                        return True
    return False

In [3]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': 'cb641a1c-522d-4a61-a575-3375ea239310', 'created_at': '2025-08-14T01:07:33.426743+00:00', 'updated_at': '2025-08-14T01:07:33.426743+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None, 'interrupts': {}, 'error': None}
cb641a1c-522d-4a61-a575-3375ea239310


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [4]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [5]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [6]:
input = { "messages": "한국 반도체 현황. 모두 한글로 만들어줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    list_chunk.append(chunk)
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])
    # elif filter_silde_data(chunk):
    #     print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
    #     print(chunk)


##########  AI 메시지  ##########
한국 반도체 현황을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 한국 반도체 산업의 역사 - 한국의 반도체 산업은 1960년대부터 시작되어 현재까지 기술 발전과 시장 확장을 거듭해왔습니다. 이 섹션에서는 초기 발전 과정과 주요 전환점을 살펴봅니다. 
3. 한국 반도체 주요 기업 현황 - 삼성전자와 SK하이닉스를 포함한 한국의 주요 반도체 기업들이 글로벌 시장에서 어떤 위치에 있는지를 조사합니다. 기업별 기술력 및 시장 점유율도 함께 분석합니다. 
4. 한국 반도체 기술 및 혁신 - 한국 반도체 산업의 기술 혁신과 최신 연구 개발 동향을 살펴봅니다. 미세 공정 기술의 발전과 차세대 반도체 기술을 포함합니다. 
5. 한국과 미국의 반도체 협력 - 한미 양국의 반도체 산업 협력 강화 방안과 그 중요성에 대해 설명합니다. 주요 협력 사례와 경제적, 기술적 이점 또한 다룹니다. 
6. 한국 반도체 제조 공정 - 전공정 및 후공정을 포함한 한국의 반도체 제조 공정에 대해 설명합니다. 웨이퍼 제조부터 패키징까지의 전체 과정을 조망합니다. 

##########  AI 메시지  ##########

    슬라이드 생성에 필요한 조사를 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드에 사용할 이미지를 검색하고 있습니다.
    
##########  AI 메시지  ##########

    슬라이드 디자인을 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드를 생성하고 있습니다.
    
##########  AI 메시지  ##########
슬라이드 생성이 완료되었습니다.


In [7]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [8]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

##########  AI 메시지  ##########
한국 반도체 현황을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 한국 반도체 산업의 역사 - 한국의 반도체 산업은 1960년대부터 시작되어 현재까지 기술 발전과 시장 확장을 거듭해왔습니다. 이 섹션에서는 초기 발전 과정과 주요 전환점을 살펴봅니다. 
3. 한국 반도체 주요 기업 현황 - 삼성전자와 SK하이닉스를 포함한 한국의 주요 반도체 기업들이 글로벌 시장에서 어떤 위치에 있는지를 조사합니다. 기업별 기술력 및 시장 점유율도 함께 분석합니다. 
4. 한국 반도체 기술 및 혁신 - 한국 반도체 산업의 기술 혁신과 최신 연구 개발 동향을 살펴봅니다. 미세 공정 기술의 발전과 차세대 반도체 기술을 포함합니다. 
5. 한국과 미국의 반도체 협력 - 한미 양국의 반도체 산업 협력 강화 방안과 그 중요성에 대해 설명합니다. 주요 협력 사례와 경제적, 기술적 이점 또한 다룹니다. 
6. 한국 반도체 제조 공정 - 전공정 및 후공정을 포함한 한국의 반도체 제조 공정에 대해 설명합니다. 웨이퍼 제조부터 패키징까지의 전체 과정을 조망합니다. 

##########  AI 메시지  ##########

    슬라이드 생성에 필요한 조사를 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드에 사용할 이미지를 검색하고 있습니다.
    
##########  AI 메시지  ##########

    슬라이드 디자인을 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드를 생성하고 있습니다.
    
##########  AI 메시지  ##########
슬라이드 생성이 완료되었습니다.


In [9]:
len(state["values"]["slides"])

7

In [10]:
state["values"]["slides"]

[{'description': '한국 반도체 현황에 대한 프레젠테이션 타이틀 페이지',
  'requirements': ['타이틀 텍스트', '부제목 또는 설명'],
  'research': True,
  'slide_type': 'title',
  'topic': '한국 반도체 현황',
  'idx': 0,
  'name': '타이틀',
  'content': "## 타이틀\n\n한국 반도체 산업은 세계 시장에서 핵심적인 역할을 수행하고 있습니다. 2023년, 한국의 반도체 수출은 특히 메모리 반도체 분야에서 중요한 기여를 하고 있습니다. 메모리 반도체는 한국의 주력 수출품목 중 하나로, 2022년 메모리 반도체가 전체 반도체 수출의 57.46%를 차지하였습니다 [1]. 그러나, 2023년에는 메모리 가격 하락 및 수요 부진으로 인해 수출이 감소하며 도전적인 한 해가 되고 있습니다 [2].\n\n한국은 2022년 세계 반도체 시장의 17.7%를 차지하며, 10년 연속 세계 2위를 유지하고 있습니다 [1]. 한국의 반도체 역량은 지속적으로 진화하고 있으며, 특히 하이엔드 제품 분야인 HBM(High Bandwidth Memory)에서는 압도적인 시장 점유율을 기록하고 있습니다 [3]. 하지만, 글로벌 공급망 불안과 주요 수출국에서의 경제 둔화와 같은 외부 요인들이 도전 과제로 작용하고 있습니다 [4].\n\n다가오는 2030년까지, 한국의 반도체 시장은 매년 8.41%의 성장률을 기록하며, 주요 산업군에서의 수요는 5G, IoT, AI 등 신기술로 인해 확대될 전망입니다 [5]. 이에 따라, 한국 반도체 기업들은 경쟁력을 유지하고자 지속적인 기술 혁신과 투자를 이어가고 있습니다.\n\n---\n\n### Sources\n\n1. [Semiconductor | InvestKOREA(ENG)](https://www.investkorea.org/ik-en/cntnts/i-312/web.do)\n2. [Korea's Semiconductor Industry: Taki

In [11]:
for s in state["values"]["slides"]:
    with open(f"{s['idx']}.html", "w", encoding="utf-8") as f:
        f.write(s['html'])